In [120]:
import cv2
import math
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
from tensorflow.keras.preprocessing import image
import numpy as np
from skimage.transform import resize

In [121]:
count = 0
videoFile = "Tom and jerry.mp4"
cap = cv2.VideoCapture(videoFile)
frameRate = cap.get(5) #frame rate
x=1
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="frame%d.jpg" % count;count+=1
        cv2.imwrite(filename, frame)
cap.release()
print ("Done!")

Done!


In [122]:
count = 0
videoFile = "Tom and Jerry 3.mp4"
cap = cv2.VideoCapture(videoFile)
frameRate = cap.get(5) #frame rate
x=1
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="test%d.jpg" % count;count+=1
        cv2.imwrite(filename, frame)
cap.release()
print ("Done!")

Done!


In [124]:
data = pd.read_csv('mapping.csv')
test = pd.read_csv('test.csv')

In [125]:
X = []
for img_name in data.Image_ID:
    img = plt.imread('' + img_name)
    X.append(img)
X = np.array(X)

test_image = []
for img_name in test.Image_ID:
    img = plt.imread('' + img_name)
    test_image.append(img)
test_img = np.array(test_image)

In [127]:
from keras.utils import to_categorical
train_y = to_categorical(data.Class)

In [141]:
image = []
for i in range(0,X.shape[0]):
    a = resize(X[i], preserve_range=True, output_shape=(224,224,3)).astype(int)
    image.append(a)
X = np.array(image)

test_image = []
for i in range(0,test_img.shape[0]):
    a = resize(test_img[i], preserve_range=True, output_shape=(224,224)).astype(int)
    test_image.append(a)
test_image = np.array(test_image)

In [168]:
X.shape

(298, 224, 224, 3)

In [142]:
from keras.applications.vgg16 import preprocess_input
X = preprocess_input(X)
test_image = preprocess_input(test_image)

In [143]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, train_y, test_size=0.3, random_state=42)

In [144]:
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout

In [145]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [146]:
X_train = base_model.predict(X_train)
X_valid = base_model.predict(X_valid)
test_image = base_model.predict(test_image)

6/6 [==============================] - 1s 125ms/step


In [147]:
X_train.shape

(208, 7, 7, 512)

In [148]:
X_train = X_train.reshape(208, 7*7*512)
X_valid = X_valid.reshape(90, 7*7*512)
test_image = test_image.reshape(186, 7*7*512)

In [149]:
train = X_train/X_train.max()
X_valid = X_valid/X_train.max()
test_image = test_image/test_image.max()

In [242]:
model = Sequential()
model.add(InputLayer((7*7*512,)))    # input layer
model.add(Dense(units=1024, activation='relu'))   # hidden layer
model.add(Dropout(0.5))      # adding dropout
model.add(Dense(units=512, activation='relu'))    # hidden layer
model.add(Dropout(0.5))      # adding dropout
model.add(Dense(units=256, activation='relu'))    # hidden layer
model.add(Dropout(0.5))      # adding dropout
model.add(Dense(3, activation='softmax'))            # output layer

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [243]:
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight
class_weights = compute_class_weight('balanced',classes = np.unique(data.Class), y = data.Class)  # computing weights of different classes
class_weights = dict(zip(np.unique(data.Class), class_weights))

In [244]:
pd.Series(data.Class).value_counts()

Class
2    120
0    111
1     67
Name: count, dtype: int64

In [245]:
from keras.callbacks import ModelCheckpoint
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]      # model check pointing based on validation loss
model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), class_weight=class_weights, callbacks=callbacks_list)

Epoch 1/100
6/7 [========================>.....] - ETA: 0s - loss: 31.1580 - accuracy: 0.4792
Epoch 1: val_loss improved from inf to 1.01753, saving model to weights.best.hdf5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


7/7 [==============================] - 3s 296ms/step - loss: 30.9653 - accuracy: 0.4856 - val_loss: 1.0175 - val_accuracy: 0.6444
Epoch 2/100
5/7 [====================>.........] - ETA: 0s - loss: 49.9707 - accuracy: 0.6000
Epoch 2: val_loss improved from 1.01753 to 0.95235, saving model to weights.best.hdf5
7/7 [==============================] - 6s 1s/step - loss: 47.8578 - accuracy: 0.5865 - val_loss: 0.9523 - val_accuracy: 0.7000
Epoch 3/100
5/7 [====================>.........] - ETA: 0s - loss: 41.6041 - accuracy: 0.7000
Epoch 3: val_loss improved from 0.95235 to 0.90300, saving model to weights.best.hdf5
7/7 [==============================] - 6s 1s/step - loss: 39.7220 - accuracy: 0.7115 - val_loss: 0.9030 - val_accuracy: 0.7333
Epoch 4/100
6/7 [========================>.....] - ETA: 0s - loss: 26.4739 - accuracy: 0.7917
Epoch 4: val_loss improved from 0.90300 to 0.87258, saving model to weights.best.hdf5
7/7 [==============================] - 1s 245ms/step - loss: 30.3892 - accur

In [246]:
model.load_weights("weights.best.hdf5")
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
scores = model.predict(test_image)

6/6 [==============================] - 0s 3ms/step


In [247]:
import numpy as np
argmaxes = []
for sublist in scores:
  argmaxes.append(np.argmax(sublist))
print(argmaxes)


[2, 2, 0, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 1, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 2, 0, 0, 2, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 1, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2]


In [249]:
time = pd.Series(argmaxes).value_counts()

In [254]:
print("Screen Time of Tom:", time[2],  "sec")
print("Screen Time of Jerry:", time[1],  "sec")
print("Screen Time neither of Tom or Jerry:", time[0],  "sec")

Screen Time of Tom: 135 sec
Screen Time of Jerry: 5 sec
Screen Time neither of Tom or Jerry: 46 sec
